In [23]:
%matplotlib notebook

### reference: https://towardsdatascience.com/attention-seq2seq-with-pytorch-learning-to-invert-a-sequence-34faf4133e53

In [24]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [25]:
torch.__version__

'1.10.2+cpu'

In [26]:
torch.__version__

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [27]:
MAX_LENGTH = 15

In [28]:
bos = 0
eos = 1
pad = 2

In [29]:
inverse_map = {
    'a':'z',
    'b':'y',
    'c':'x',
    'd':'w',
    'e':'v',
    'f':'u',
    'g':'t',
    'h':'s',
    'i':'r',
    'j':'q',
    'k':'p',
    'l':'o',
    'm':'n',
    'n':'m',
    'o':'l',
    'p':'k',
    'q':'j',
    'r':'i',
    's':'h',
    't':'g',
    'u':'f',
    'v':'e',
    'w':'d',
    'x':'c',
    'y':'b',
    'z':'a'
}

In [30]:
a2i = {
    '<s>':0,
    '</s>':1,
    '<pad>':2,
    'a':3,
    'b':4,
    'c':5,
    'd':6,
    'e':7,
    'f':8,
    'g':9,
    'h':10,
    'i':11,
    'j':12,
    'k':13,
    'l':14,
    'm':15,
    'n':16,
    'o':17,
    'p':18,
    'q':19,
    'r':20,
    's':21,
    't':22,
    'u':23,
    'v':24,
    'w':25,
    'x':26,
    'y':27,
    'z':28,
}

In [31]:
i2a = {v:k for k, v in a2i.items()}

In [32]:
i2a

{0: '<s>',
 1: '</s>',
 2: '<pad>',
 3: 'a',
 4: 'b',
 5: 'c',
 6: 'd',
 7: 'e',
 8: 'f',
 9: 'g',
 10: 'h',
 11: 'i',
 12: 'j',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'q',
 20: 'r',
 21: 's',
 22: 't',
 23: 'u',
 24: 'v',
 25: 'w',
 26: 'x',
 27: 'y',
 28: 'z'}

In [33]:
def generate_random_alphabet_index():
    random_length = np.random.randint(10, MAX_LENGTH-2)    # -2 because of <s> and </s>
    random_alphabet_index = np.random.randint(0, 26, random_length) + 3
    return random_alphabet_index.tolist(), random_length

In [34]:
def make_alphabet_tensor():
    indice, length = generate_random_alphabet_index()
    indice = indice + [pad]*(MAX_LENGTH-length)
    return torch.tensor([indice]).to(device)

In [35]:
class AlphabetToyDataset(Dataset):
    def __init__(self, n_dataset=1000):
        bos = 0
        eos = 1
        pad = 2
        self.inputs = []
        self.labels = []
        self.length = []
        for _ in range(n_dataset):
            # make input example
            aindex, alen = generate_random_alphabet_index()
            
            # index to alphabet
            #alphabet = list(map(lambda a: i2a[a], aindex))
            
            # inversing
            #inversed_alphabet = list(map(lambda a: inverse_map[a], alphabet))
            
            # alphabet to index
            #iindex = list(map(lambda ia: a2i[ia], inversed_alphabet))
            iindex = aindex[::-1]
            
            # add bos, eos and pad
            n_pad = MAX_LENGTH - len(aindex) - 1
            aindex = aindex + [eos] + [pad]*n_pad
            iindex = iindex + [eos] + [pad]*n_pad
            
            # add to examples
            self.inputs.append(aindex)
            self.labels.append(iindex)
            self.length.append(alen)
            
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return [
            torch.tensor(self.inputs[index], dtype=torch.long),
            torch.tensor(self.labels[index], dtype=torch.long),
            torch.tensor(self.length[index], dtype=torch.long)
        ]

In [36]:
# class AlphabetToyDataset(Dataset):
#     def __init__(self, n_dataset=1000):
#         bos = 0
#         eos = 1
#         pad = 2
#         self.inputs = []
#         self.labels = []
#         for _ in range(n_dataset):
#             # make input example
#             aindex = generate_random_alphabet_index()
            
#             # index to alphabet
#             alphabet = list(map(lambda a: i2a[a], aindex))
            
#             # inversing
#             inversed_alphabet = list(map(lambda a: inverse_map[a], alphabet))
            
#             # alphabet to index
#             iindex = list(map(lambda ia: a2i[ia], inversed_alphabet))
            
#             # add bos, eos and pad
#             n_pad = MAX_LENGTH - len(aindex) - 2
#             aindex = aindex + [eos]
#             iindex = iindex + [eos]
            
#             # add to examples
#             self.inputs.append(aindex)
#             self.labels.append(iindex)
            
#     def __len__(self):
#         return len(self.inputs)
    
#     def __getitem__(self, index):
#         return [
#             torch.tensor(self.inputs[index], dtype=torch.long),
#             torch.tensor(self.labels[index], dtype=torch.long)
#         ]

In [37]:
train_dataset = AlphabetToyDataset(n_dataset=3000)
valid_dataset = AlphabetToyDataset(n_dataset=300)

In [38]:
len(train_dataset), len(valid_dataset)

(3000, 300)

In [39]:
def convert_index_to_alphabet(index):
    alphabet = list(map(lambda i: i2a[i], index))
    return ' '.join(alphabet)

In [40]:
for i in range(3):
    ex = train_dataset[i]
    aindex, iindex, length = ex
    
    print('aindex_{}: {}'.format(len(aindex), convert_index_to_alphabet(aindex.numpy())))
    print('iindex_{}: {}'.format(len(iindex), convert_index_to_alphabet(iindex.numpy())))
    print('** aindex_{}: {}'.format(len(aindex), aindex))
    print('** iindex_{}: {}'.format(len(iindex), iindex))
    print('------------')

aindex_15: k e i l l o t q s s </s> <pad> <pad> <pad> <pad>
iindex_15: s s q t o l l i e k </s> <pad> <pad> <pad> <pad>
** aindex_15: tensor([13,  7, 11, 14, 14, 17, 22, 19, 21, 21,  1,  2,  2,  2,  2])
** iindex_15: tensor([21, 21, 19, 22, 17, 14, 14, 11,  7, 13,  1,  2,  2,  2,  2])
------------
aindex_15: n f y m y j y d e k t j </s> <pad> <pad>
iindex_15: j t k e d y j y m y f n </s> <pad> <pad>
** aindex_15: tensor([16,  8, 27, 15, 27, 12, 27,  6,  7, 13, 22, 12,  1,  2,  2])
** iindex_15: tensor([12, 22, 13,  7,  6, 27, 12, 27, 15, 27,  8, 16,  1,  2,  2])
------------
aindex_15: y p h l u h t t z s b b </s> <pad> <pad>
iindex_15: b b s z t t h u l h p y </s> <pad> <pad>
** aindex_15: tensor([27, 18, 10, 14, 23, 10, 22, 22, 28, 21,  4,  4,  1,  2,  2])
** iindex_15: tensor([ 4,  4, 21, 28, 22, 22, 10, 23, 14, 10, 18, 27,  1,  2,  2])
------------


In [41]:
def collate_fn(batch):
    inputs = pad_sequence([b[0] for b in batch], batch_first=True)
    targets = pad_sequence([b[1] for b in batch], batch_first=True)
    lengths = torch.stack([b[2] for b in batch])

    lengths, indice = torch.sort(lengths, descending=True)
    inputs = inputs[indice]
    targets = targets[indice]
    return inputs, targets, lengths

In [42]:
train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=16)
valid_dataloader = DataLoader(valid_dataset, collate_fn=collate_fn, batch_size=1)

In [43]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, inputs, hidden):
        '''
        Input Parameters
        - inputs: (B,M)
        - hidden: (1,B,H)
        
        Output returns
        - output: (B,M,H)
        - hidden: (1,B,H)
        
        Logging outputs
        ** inputs: torch.Size([16, 5])
        ** hidden: torch.Size([1, 16, 256])
        ** embedded: torch.Size([16, 5, 256])
        ** final-output: torch.Size([16, 5, 256])
        ** final-hidden: torch.Size([1, 16, 256])
        '''
        #print('** inputs: {}'.format(inputs.shape))
        #print('** hidden: {}'.format(hidden.shape))
        embedded = self.embedding(inputs)    # (B,M,H)
        #print('** embedded: {}'.format(embedded.shape))
        output, hidden = self.gru(embedded, hidden)    # (B,M,H), (1,B,H)
        #print('** final-output: {}'.format(output.shape))
        #print('** final-hidden: {}'.format(hidden.shape))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [44]:
hidden_size = 256
enc = EncoderRNN(26+3, hidden_size).to(device)

In [45]:
inp = torch.from_numpy(np.array([
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
]))
inp = inp.to(device)
hid = enc.initHidden(inp.size(0))

In [46]:
o, h = enc(inp, hid)
o.shape, h.shape

(torch.Size([16, 5, 256]), torch.Size([1, 16, 256]))

In [47]:
# without Attention
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #print('DecoderRNN_INPUT: input: {} → {:.4f}'.format(input.shape, input.sum()))
        #print('DecoderRNN_INPUT: hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        #print('** output: {} → {:.4f}'.format(output.shape, output.sum()))
        
        
        output, hidden = self.gru(output, hidden)
        #print('** gru-output: {} → {:.4f}'.format(output.shape, output.sum()))
        #print('** gru-hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        
        output = self.softmax(self.out(output[0]))
        #print('** hidden: {}'.format(hidden.shape))
        
        #print('DecoderRNN_OUTPUT: output: {} → {:.4f}'.format(output.shape, output.sum()))
        #print('DecoderRNN_OUTPUT: hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [48]:
# with attention
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, inputs, hidden, encoder_outputs):
        '''
        Input Parameters
        - inputs: (B,1)
        - hidden: (1,B,H)
        - encoder_outputs: (B,M,H)
        
        Output returns
        - output: (B,1,O)
        - hidden: (B,1,H)
        - attn_weights: (B,1,M)
        
        Logging outputs
        ** inputs: torch.Size([16, 1])
        ** hidden: torch.Size(v[1, 16, 256])
        ** encoder_outputs: torch.Size([16, 15, 256])
        ** embedded: torch.Size([16, 1, 256])
        ** attn_weights: torch.Size([16, 1, 15])
        ** attn_weights: tensor([[[0.1068, 0.0260, 0.0466, 0.0421, 0.0885, 0.0843, 0.0718, 0.0575,
                  0.0361, 0.0273, 0.0339, 0.1882, 0.0606, 0.0723, 0.0581]],

                [[0.1634, 0.0374, 0.0518, 0.0382, 0.0771, 0.0577, 0.0624, 0.0658,
                  0.0286, 0.0351, 0.0336, 0.1435, 0.0603, 0.0940, 0.0510]],

                [[0.1191, 0.0284, 0.0447, 0.0361, 0.0816, 0.0879, 0.0675, 0.0523,
                  0.0293, 0.0273, 0.0535, 0.1981, 0.0588, 0.0728, 0.0427]],

                [[0.1043, 0.0357, 0.0479, 0.0426, 0.1006, 0.0658, 0.0556, 0.0538,
                  0.0372, 0.0476, 0.0348, 0.1877, 0.0641, 0.0705, 0.0516]],

                [[0.1179, 0.0354, 0.0404, 0.0565, 0.0785, 0.0645, 0.0700, 0.0685,
                  0.0380, 0.0414, 0.0456, 0.1519, 0.0710, 0.0682, 0.0523]],

                [[0.1363, 0.0500, 0.0404, 0.0383, 0.0748, 0.0674, 0.0733, 0.0520,
                  0.0288, 0.0306, 0.0466, 0.1517, 0.0769, 0.0932, 0.0395]],

                [[0.1429, 0.0315, 0.0380, 0.0411, 0.0754, 0.0555, 0.0701, 0.0626,
                  0.0337, 0.0311, 0.0398, 0.1486, 0.0813, 0.0885, 0.0599]],

                [[0.1183, 0.0333, 0.0422, 0.0445, 0.0842, 0.0600, 0.0854, 0.0732,
                  0.0299, 0.0447, 0.0299, 0.1091, 0.0865, 0.0831, 0.0757]],

                [[0.1145, 0.0353, 0.0372, 0.0386, 0.0867, 0.0568, 0.0840, 0.0514,
                  0.0393, 0.0285, 0.0397, 0.1800, 0.0759, 0.0755, 0.0565]],

                [[0.1614, 0.0251, 0.0357, 0.0470, 0.0903, 0.0756, 0.0521, 0.0419,
                  0.0305, 0.0227, 0.0343, 0.1683, 0.0643, 0.0891, 0.0616]],

                [[0.1152, 0.0248, 0.0413, 0.0535, 0.0836, 0.0599, 0.0707, 0.0559,
                  0.0333, 0.0394, 0.0348, 0.1688, 0.0728, 0.0880, 0.0582]],

                [[0.1475, 0.0310, 0.0462, 0.0446, 0.0865, 0.0484, 0.0775, 0.0775,
                  0.0378, 0.0357, 0.0360, 0.1070, 0.0689, 0.1070, 0.0485]],

                [[0.1168, 0.0299, 0.0347, 0.0407, 0.0787, 0.0605, 0.0683, 0.0517,
                  0.0312, 0.0334, 0.0426, 0.1895, 0.0739, 0.0914, 0.0565]],

                [[0.1122, 0.0315, 0.0371, 0.0409, 0.1061, 0.0805, 0.0861, 0.0475,
                  0.0399, 0.0262, 0.0330, 0.1467, 0.0676, 0.0788, 0.0660]],

                [[0.1491, 0.0363, 0.0410, 0.0398, 0.0919, 0.0586, 0.0605, 0.0647,
                  0.0378, 0.0277, 0.0356, 0.1396, 0.0699, 0.0981, 0.0495]],

                [[0.1127, 0.0238, 0.0486, 0.0385, 0.0809, 0.0726, 0.0829, 0.0495,
                  0.0441, 0.0275, 0.0412, 0.1654, 0.0627, 0.0825, 0.0672]]],
               device='cuda:0', grad_fn=<SoftmaxBackward>)
        ** attn_weights.sum(): torch.Size([])
        ** attn_weights.sum(): 16.0000
        ** attn_applied: torch.Size([16, 1, 256])
        ** output: torch.Size([16, 1, 512])
        ** output: torch.Size([16, 1, 256])
        ** output: torch.Size([16, 1, 256])
        ** hidden: torch.Size([1, 16, 256])
        ** gru-output: torch.Size([16, 1, 256])
        ** gru-hidden: torch.Size([1, 16, 256])
        ** final-output: torch.Size([16, 1, 29])
        ** final-hidden: torch.Size([1, 16, 256])
        ** final-attn_weights: torch.Size([16, 1, 15])
        '''
        embedded = self.embedding(inputs)    # (B,1,H)
        embedded = self.dropout(embedded)
        
        # query: embedded
        # key: hidden
        # value: encoder_outputs
        
        attn_weights = F.softmax(
            self.attn(
                torch.cat((embedded, hidden.transpose(0, 1)), -1)    # (B,1,2H)
            ),    # (B,1,M)
            dim=-1)    # (B,1,M)
        #print('** attn_weights: {}'.format(attn_weights.shape))
        #print('** attn_weights: {}'.format(attn_weights))
        #print('** attn_weights.sum(): {}'.format(attn_weights.sum().shape))
        #print('** attn_weights.sum(): {:.4f}'.format(attn_weights.sum()))
        
        attn_applied = torch.bmm(attn_weights, encoder_outputs)    # (B,1,H)
        #print('** attn_applied: {}'.format(attn_applied.shape))
        #return

        output = torch.cat((embedded, attn_applied), -1)    # (B,1,2H)
        #print('** output: {}'.format(output.shape))
        output = self.attn_combine(output)    # (B,1,H)
        #print('** output: {}'.format(output.shape))    # (B,1,H)
        
        output = F.relu(output)
        #print('** output: {}'.format(output.shape))    # (B,1,H)
        #print('** hidden: {}'.format(hidden.shape))    # (B,1,H)
        output, hidden = self.gru(output, hidden)    # (B,1,H) (1,B,H)
        #print('** gru-output: {}'.format(output.shape))
        #print('** gru-hidden: {}'.format(hidden.shape))

        output = F.log_softmax(self.out(output), dim=-1)
        #print('** final-output: {}'.format(output.shape))
        #print('** final-hidden: {}'.format(hidden.shape))
        #print('** final-attn_weights: {}'.format(attn_weights.shape))
        return output, hidden, attn_weights

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [49]:
teacher_forcing_ratio = 0.5

In [50]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH, with_attention=True):
    batch_size = input_tensor.size(0)
    encoder_hidden = encoder.initHidden(batch_size)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
    
    decoder_input = torch.tensor([bos]*batch_size, device=device)
    decoder_input = decoder_input.unsqueeze(-1)    # (B,1)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if np.random.random() < teacher_forcing_ratio else False
    
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input,    # (B,1)
            decoder_hidden,   # (1,B,H)
            encoder_outputs   # (B,M,H)
        )
        decoder_output = decoder_output.squeeze(1)
        
        loss += criterion(decoder_output, target_tensor[:,di])
        
        if use_teacher_forcing:
            decoder_input = target_tensor[:,di].unsqueeze(-1)    # (B,1)
        else:
            topv, topi = decoder_output.topk(1)
            decoder_input = topi    # (B,1)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [51]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [52]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, with_attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter, batch in enumerate(train_dataloader):
        input_tensor, target_tensor, length_tensor = batch
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)
        length_tensor = length_tensor.to(device)
        
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, with_attention=with_attention)
        print_loss_total += loss
        plot_loss_total += loss

        if (iter+1) % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, (iter+1) / n_iters),
                                         (iter+1), (iter+1) / n_iters * 100, print_loss_avg))

        if (iter+1) % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [53]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [54]:
def evaluate(encoder, decoder, input_tensor, max_length=MAX_LENGTH):
    with torch.no_grad():
        batch_size = input_tensor.size(0)
        encoder_hidden = encoder.initHidden(batch_size)
        #print(input_tensor.shape, encoder_hidden.shape)
        encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
        
        decoder_input = torch.tensor([bos]*batch_size, device=device)
        decoder_input = decoder_input.unsqueeze(-1)    # (B,1)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            #print('** ', decoder_input.shape, decoder_hidden.shape, encoder_outputs.shape)
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input,    # (B,1)
                decoder_hidden,   # (1,B,H)
                encoder_outputs   # (B,M,H)
            )
            decoder_output = decoder_output.squeeze(1)
            decoder_attentions[:, di] = decoder_attention
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi    # (B,1)
            if topi.item() == eos:
                decoded_words.append('</s>')
                break
            else:
                decoded_words.append(i2a[topi.item()])

        return decoded_words, decoder_attentions[:di + 1]

In [55]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [56]:
for t in train_dataloader:
    print(t)
    break

(tensor([[16,  8, 27, 15, 27, 12, 27,  6,  7, 13, 22, 12,  1,  2,  2],
        [27, 18, 10, 14, 23, 10, 22, 22, 28, 21,  4,  4,  1,  2,  2],
        [16, 23, 27, 22, 13,  4,  4, 26,  5,  8,  6, 15,  1,  2,  2],
        [21, 14,  8, 17,  7, 15, 17, 19,  8, 23,  3, 16,  1,  2,  2],
        [15, 28, 25,  4, 23, 25,  6, 25, 26, 21, 18, 19,  1,  2,  2],
        [ 5, 13,  8, 12, 11,  8, 23, 13,  5, 22, 28, 24,  1,  2,  2],
        [26,  3,  6, 17,  9, 15, 12,  6, 15, 19, 18, 18,  1,  2,  2],
        [25, 14, 16, 19, 17, 23,  4, 17, 23, 26, 18,  3,  1,  2,  2],
        [17, 21, 25, 22, 17, 16,  4,  8, 13, 17,  6,  1,  2,  2,  2],
        [ 6,  3, 28,  6, 22, 11, 19, 12, 19, 17,  3,  1,  2,  2,  2],
        [17, 20,  4, 12, 11, 12, 13,  6, 16, 16,  6,  1,  2,  2,  2],
        [18, 14, 27,  7,  7,  9, 19, 10, 18, 27,  5,  1,  2,  2,  2],
        [13,  7, 11, 14, 14, 17, 22, 19, 21, 21,  1,  2,  2,  2,  2],
        [ 8, 11,  6, 22, 17, 13,  5, 13, 25, 24,  1,  2,  2,  2,  2],
        [28,  5,  4

In [57]:
hidden_size = 256
encoder1 = EncoderRNN(26+3, hidden_size).to(device)
decoder1 = AttnDecoderRNN(hidden_size, 26+3, dropout_p=0.1).to(device)

In [58]:
# hidden_size = 256
# encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

In [59]:
#### Input validation

In [60]:
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [61]:
for _ in range(15):
    trainIters(encoder1, decoder1, 75000, print_every=30, with_attention=True)

0m 1s (- 70m 31s) (30 0%) 3.0902
0m 3s (- 67m 38s) (60 0%) 2.8967
0m 4s (- 67m 12s) (90 0%) 2.8707
0m 6s (- 67m 28s) (120 0%) 2.7985
0m 8s (- 66m 32s) (150 0%) 2.7116
0m 9s (- 65m 53s) (180 0%) 2.7236
0m 1s (- 66m 56s) (30 0%) 2.7000
0m 3s (- 66m 42s) (60 0%) 2.6320
0m 4s (- 69m 6s) (90 0%) 2.6358
0m 6s (- 70m 43s) (120 0%) 2.6157
0m 8s (- 70m 44s) (150 0%) 2.5887
0m 10s (- 71m 23s) (180 0%) 2.6088
0m 1s (- 63m 53s) (30 0%) 2.5476
0m 3s (- 67m 10s) (60 0%) 2.5378
0m 4s (- 66m 49s) (90 0%) 2.5472
0m 6s (- 66m 39s) (120 0%) 2.5287
0m 7s (- 66m 25s) (150 0%) 2.5388
0m 9s (- 67m 7s) (180 0%) 2.4403
0m 1s (- 68m 11s) (30 0%) 2.4282
0m 3s (- 65m 19s) (60 0%) 2.4278
0m 4s (- 64m 38s) (90 0%) 2.3940
0m 6s (- 64m 49s) (120 0%) 2.2744
0m 7s (- 65m 20s) (150 0%) 2.2214
0m 9s (- 65m 4s) (180 0%) 2.1003
0m 1s (- 70m 58s) (30 0%) 1.8535
0m 3s (- 70m 34s) (60 0%) 1.7906
0m 4s (- 68m 28s) (90 0%) 1.6020
0m 6s (- 69m 57s) (120 0%) 1.4154
0m 8s (- 68m 31s) (150 0%) 1.2278
0m 10s (- 69m 19s) (180 0%) 1.0

C:\Users\jinki.lee\AppData\Local\Temp\ipykernel_14240\734353137.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


0m 1s (- 75m 0s) (30 0%) 0.1342
0m 3s (- 72m 20s) (60 0%) 0.1043
0m 5s (- 72m 38s) (90 0%) 0.1292
0m 6s (- 71m 31s) (120 0%) 0.1251
0m 8s (- 71m 0s) (150 0%) 0.1055
0m 10s (- 70m 31s) (180 0%) 0.1021
0m 1s (- 72m 53s) (30 0%) 0.1042
0m 3s (- 71m 6s) (60 0%) 1.1068
0m 5s (- 75m 54s) (90 0%) 0.1704
0m 7s (- 78m 46s) (120 0%) 0.1378
0m 9s (- 79m 6s) (150 0%) 0.1182
0m 11s (- 79m 19s) (180 0%) 0.1198
0m 2s (- 86m 14s) (30 0%) 0.0974
0m 5s (- 109m 35s) (60 0%) 0.0949
0m 7s (- 109m 50s) (90 0%) 0.1018
0m 10s (- 106m 39s) (120 0%) 0.0972
0m 12s (- 101m 56s) (150 0%) 0.0797
0m 14s (- 97m 45s) (180 0%) 0.0919
0m 1s (- 75m 20s) (30 0%) 0.0917
0m 3s (- 75m 50s) (60 0%) 0.0744
0m 5s (- 77m 50s) (90 0%) 0.0900
0m 7s (- 78m 1s) (120 0%) 0.0837
0m 9s (- 76m 39s) (150 0%) 0.0701
0m 10s (- 75m 48s) (180 0%) 0.0890


In [62]:
for _ in range(5):
    trainIters(encoder1, decoder1, 75000, print_every=30, with_attention=True)

0m 1s (- 82m 35s) (30 0%) 0.0752
0m 3s (- 79m 38s) (60 0%) 0.0602
0m 5s (- 76m 15s) (90 0%) 0.0775
0m 7s (- 75m 29s) (120 0%) 0.0658
0m 8s (- 74m 15s) (150 0%) 0.0682
0m 10s (- 76m 8s) (180 0%) 0.0841
0m 1s (- 76m 45s) (30 0%) 0.0668
0m 3s (- 73m 33s) (60 0%) 0.0670
0m 5s (- 77m 42s) (90 0%) 0.0611
0m 7s (- 79m 17s) (120 0%) 0.0683
0m 9s (- 79m 23s) (150 0%) 0.0531
0m 11s (- 79m 53s) (180 0%) 0.0662
0m 2s (- 84m 15s) (30 0%) 0.0527
0m 4s (- 84m 21s) (60 0%) 0.0589
0m 5s (- 80m 26s) (90 0%) 0.0639
0m 7s (- 82m 6s) (120 0%) 0.0696
0m 11s (- 91m 59s) (150 0%) 0.0631
0m 13s (- 92m 15s) (180 0%) 0.0576
0m 2s (- 85m 12s) (30 0%) 0.0645
0m 4s (- 83m 44s) (60 0%) 0.0433
0m 7s (- 99m 40s) (90 0%) 0.0566
0m 9s (- 96m 9s) (120 0%) 0.0676
0m 11s (- 91m 33s) (150 0%) 0.0460
0m 12s (- 88m 47s) (180 0%) 0.0449
0m 1s (- 75m 50s) (30 0%) 0.0419
0m 3s (- 75m 57s) (60 0%) 0.0446
0m 5s (- 78m 16s) (90 0%) 0.0440
0m 7s (- 78m 56s) (120 0%) 0.0498
0m 9s (- 77m 21s) (150 0%) 0.0394
0m 10s (- 75m 53s) (180 0%

In [63]:
# trainIters(encoder1, decoder1, 75000, print_every=100, with_attention=True)

In [64]:
for d in valid_dataloader:
    print(d[0].shape)
    break

torch.Size([1, 15])


In [65]:
for d in valid_dataloader:
    input_tensor, target_tensor, length_tensor = d
    input_string = list(map(lambda i: i2a[i], input_tensor.numpy()[0]))
    output_string = list(map(lambda i: i2a[i], target_tensor.numpy()[0]))
    input_string = np.array(input_string)
    output_string = np.array(output_string)
    print('input: {}'.format(input_string))
    print('output: {}'.format(output_string))
    
    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(device)
    length_tensor = length_tensor.to(device)
    
    pred, attn_weight = evaluate(encoder1, decoder1, input_tensor)
    pred = np.array(pred)
    print('pred: {}'.format(pred))
    print('-------------')
    #break

input: ['l' 'a' 'u' 's' 'l' 's' 'f' 'n' 'v' 't' 'd' 'l' '</s>' '<pad>' '<pad>']
output: ['l' 'd' 't' 'v' 'n' 'f' 's' 'l' 's' 'u' 'a' 'l' '</s>' '<pad>' '<pad>']
pred: ['l' 'd' 't' 'v' 'n' 'f' 's' 'l' 's' 'u' 'a' 'l' '</s>']
-------------
input: ['a' 'e' 'n' 'y' 'i' 'e' 'e' 'a' 'y' 'f' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['f' 'y' 'a' 'e' 'e' 'i' 'y' 'n' 'e' 'a' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['f' 'y' 'a' 'e' 'e' 'i' 'y' 'n' 'e' 'a' '</s>']
-------------
input: ['e' 's' 'j' 'b' 'n' 'w' 'e' 'j' 'c' 'd' 'd' 'n' '</s>' '<pad>' '<pad>']
output: ['n' 'd' 'd' 'c' 'j' 'e' 'w' 'n' 'b' 'j' 's' 'e' '</s>' '<pad>' '<pad>']
pred: ['n' 'd' 'd' 'c' 'j' 'e' 'w' 'n' 'b' 'j' 's' 'e' '</s>']
-------------
input: ['t' 'n' 't' 'e' 'g' 't' 'c' 'y' 't' 't' 'a' 's' '</s>' '<pad>' '<pad>']
output: ['s' 'a' 't' 't' 'y' 'c' 't' 'g' 'e' 't' 'n' 't' '</s>' '<pad>' '<pad>']
pred: ['s' 'a' 's' 't' 'y' 'c' 't' 'g' 'e' 't' 'n' 't' '</s>']
-------------
input: ['n' 't' 'b' 'x' 'x' 'm' 'l' 'h

input: ['n' 'k' 'p' 'z' 'v' 'q' 'v' 'y' 'e' 'i' 'g' 'y' '</s>' '<pad>' '<pad>']
output: ['y' 'g' 'i' 'e' 'y' 'v' 'q' 'v' 'z' 'p' 'k' 'n' '</s>' '<pad>' '<pad>']
pred: ['y' 'g' 'y' 'e' 'y' 'v' 'q' 'v' 'z' 'p' 'k' 'n' '</s>']
-------------
input: ['h' 'a' 's' 'n' 'v' 'm' 'r' 'b' 'x' 'q' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['q' 'x' 'b' 'r' 'm' 'v' 'n' 's' 'a' 'h' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['q' 'x' 'b' 'r' 'm' 'v' 'n' 's' 'a' 'h' '</s>']
-------------
input: ['z' 'a' 'w' 'b' 'c' 'i' 'z' 'a' 'r' 'c' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['c' 'r' 'a' 'z' 'i' 'c' 'b' 'w' 'a' 'z' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['c' 'r' 'a' 'z' 'i' 'c' 'b' 'w' 'a' 'z' '</s>']
-------------
input: ['s' 'r' 'b' 'q' 'd' 'a' 'v' 'y' 'b' 'q' 'r' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['r' 'q' 'b' 'y' 'v' 'a' 'd' 'q' 'b' 'r' 's' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['r' 'q' 'b' 'y' 'v' 'a' 'd' 'q' 'b' 'r' 's' '</s>']
-------------
input: ['m' 'b' 'c' 'o

input: ['b' 'm' 'i' 'j' 'i' 'd' 'q' 'u' 'o' 'b' 'i' 'w' '</s>' '<pad>' '<pad>']
output: ['w' 'i' 'b' 'o' 'u' 'q' 'd' 'i' 'j' 'i' 'm' 'b' '</s>' '<pad>' '<pad>']
pred: ['w' 'i' 'b' 'o' 'u' 'q' 'd' 'i' 'j' 'i' 'm' 'b' '</s>']
-------------
input: ['f' 'f' 'r' 'b' 'i' 'z' 'h' 'c' 'a' 's' 'o' 'a' '</s>' '<pad>' '<pad>']
output: ['a' 'o' 's' 'a' 'c' 'h' 'z' 'i' 'b' 'r' 'f' 'f' '</s>' '<pad>' '<pad>']
pred: ['a' 'o' 'a' 'a' 'c' 'h' 'z' 'i' 'b' 'r' 'f' 'f' '</s>']
-------------
input: ['l' 's' 'y' 'c' 'y' 'e' 'z' 'i' 'e' 'c' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['c' 'e' 'i' 'z' 'e' 'y' 'c' 'y' 's' 'l' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['c' 'e' 'i' 'z' 'e' 'y' 'c' 'y' 's' 'l' '</s>']
-------------
input: ['h' 'q' 't' 'f' 'h' 'r' 'e' 'a' 'h' 'n' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['n' 'h' 'a' 'e' 'r' 'h' 'f' 't' 'q' 'h' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['n' 'h' 'a' 'e' 'r' 'h' 'f' 't' 'q' 'h' '</s>']
-------------
input: ['h' 'x' 'p' 'i' 'm' 

input: ['o' 'c' 'z' 'c' 'n' 'j' 'x' 'l' 'g' 'a' 'e' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['e' 'a' 'g' 'l' 'x' 'j' 'n' 'c' 'z' 'c' 'o' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['e' 'a' 'g' 'l' 'x' 'j' 'n' 'c' 'z' 'c' 'o' '</s>']
-------------
input: ['n' 'v' 'm' 'k' 'f' 'x' 'y' 'u' 't' 'x' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['x' 't' 'u' 'y' 'x' 'f' 'k' 'm' 'v' 'n' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['x' 't' 'u' 'y' 'x' 'f' 'k' 'm' 'v' 'n' '</s>']
-------------
input: ['e' 'x' 'm' 'h' 'o' 'k' 'p' 't' 'f' 'd' 'y' 'o' '</s>' '<pad>' '<pad>']
output: ['o' 'y' 'd' 'f' 't' 'p' 'k' 'o' 'h' 'm' 'x' 'e' '</s>' '<pad>' '<pad>']
pred: ['o' 'y' 'o' 'f' 't' 'p' 'k' 'o' 'h' 'm' 'x' 'e' '</s>']
-------------
input: ['a' 'j' 'z' 'v' 'u' 'h' 'g' 'x' 'b' 'i' 'q' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['q' 'i' 'b' 'x' 'g' 'h' 'u' 'v' 'z' 'j' 'a' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['q' 'i' 'b' 'x' 'g' 'h' 'u' 'v' 'z' 'j' 'a' '</s>']
-------------
input: ['j' 'p' 'u' 'j' 'h

input: ['k' 'x' 'u' 'n' 'z' 'k' 'e' 'b' 'd' 'i' 'x' 'j' '</s>' '<pad>' '<pad>']
output: ['j' 'x' 'i' 'd' 'b' 'e' 'k' 'z' 'n' 'u' 'x' 'k' '</s>' '<pad>' '<pad>']
pred: ['j' 'x' 'j' 'd' 'b' 'e' 'k' 'z' 'n' 'u' 'x' 'k' '</s>']
-------------
input: ['h' 'e' 'c' 'f' 'g' 'z' 'c' 'q' 'b' 'h' 'u' 'c' '</s>' '<pad>' '<pad>']
output: ['c' 'u' 'h' 'b' 'q' 'c' 'z' 'g' 'f' 'c' 'e' 'h' '</s>' '<pad>' '<pad>']
pred: ['c' 'u' 'c' 'b' 'q' 'c' 'z' 'g' 'f' 'c' 'e' 'h' '</s>']
-------------
input: ['u' 'q' 'd' 'a' 'n' 'o' 'q' 'f' 'p' 'c' 'n' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['n' 'c' 'p' 'f' 'q' 'o' 'n' 'a' 'd' 'q' 'u' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['n' 'c' 'p' 'f' 'q' 'o' 'n' 'a' 'd' 'q' 'u' '</s>']
-------------
input: ['z' 'r' 'y' 'e' 'b' 'c' 'n' 'b' 'q' 't' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['t' 'q' 'b' 'n' 'c' 'b' 'e' 'y' 'r' 'z' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['t' 'q' 'b' 'n' 'c' 'b' 'e' 'y' 'r' 'z' '</s>']
-------------
input: ['a' 'x' 'p' 'g' 's' 't' 

pred: ['l' 'n' 'x' 't' 'u' 'p' 'd' 'w' 'o' 't' '</s>']
-------------
input: ['h' 'r' 'i' 'f' 'g' 'w' 'f' 'l' 'w' 'l' 'g' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['g' 'l' 'w' 'l' 'f' 'w' 'g' 'f' 'i' 'r' 'h' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['g' 'l' 'w' 'l' 'f' 'w' 'g' 'f' 'i' 'r' 'h' '</s>']
-------------
input: ['z' 'c' 'b' 'a' 'z' 'x' 'd' 'n' 'v' 'e' 'x' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['x' 'e' 'v' 'n' 'd' 'x' 'z' 'a' 'b' 'c' 'z' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['x' 'e' 'v' 'n' 'd' 'x' 'z' 'a' 'b' 'c' 'z' '</s>']
-------------
input: ['e' 'm' 'j' 'z' 'u' 'y' 'n' 'c' 'f' 'h' 'q' 'd' '</s>' '<pad>' '<pad>']
output: ['d' 'q' 'h' 'f' 'c' 'n' 'y' 'u' 'z' 'j' 'm' 'e' '</s>' '<pad>' '<pad>']
pred: ['d' 'q' 'q' 'f' 'c' 'n' 'y' 'u' 'z' 'j' 'm' 'e' '</s>']
-------------
input: ['g' 's' 'g' 'c' 'c' 'e' 't' 'r' 'z' 'p' 'l' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['l' 'p' 'z' 'r' 't' 'e' 'c' 'c' 'g' 's' 'g' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['l' 'p' 'z' 'r' 't' 'e' 'c'

pred: ['b' 'c' 'c' 'd' 'b' 'f' 'm' 'g' 'k' 'i' '</s>']
-------------
input: ['i' 'd' 's' 'j' 'g' 'd' 'y' 'i' 'o' 'y' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['y' 'o' 'i' 'y' 'd' 'g' 'j' 's' 'd' 'i' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['y' 'o' 'i' 'y' 'd' 'g' 'j' 's' 'd' 'i' '</s>']
-------------
input: ['m' 'k' 'm' 'f' 'i' 'h' 'g' 'b' 'b' 'r' 'r' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['r' 'r' 'b' 'b' 'g' 'h' 'i' 'f' 'm' 'k' 'm' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['r' 'b' 'b' 'b' 'g' 'h' 'i' 'f' 'm' 'k' 'm' '</s>']
-------------
input: ['v' 'p' 'f' 'q' 'j' 'v' 'z' 'a' 'f' 'b' 'x' 'f' '</s>' '<pad>' '<pad>']
output: ['f' 'x' 'b' 'f' 'a' 'z' 'v' 'j' 'q' 'f' 'p' 'v' '</s>' '<pad>' '<pad>']
pred: ['f' 'x' 'f' 'f' 'a' 'z' 'v' 'j' 'q' 'f' 'p' 'v' '</s>']
-------------
input: ['i' 'y' 'b' 'f' 'i' 's' 's' 'q' 'l' 'e' 'y' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['y' 'e' 'l' 'q' 's' 's' 'i' 'f' 'b' 'y' 'i' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['y' 'e' 'l' 'q' 's' 's'

In [66]:
# plt.matshow(attn_weight.numpy())

In [67]:
attn_weight.shape

torch.Size([12, 15])

In [68]:
input_tensor = make_alphabet_tensor()
output_words, attentions = evaluate(encoder1, decoder1, input_tensor)

In [69]:
def showAttention(input_sentence, output_words, attentions):
    # colorbar로 그림 설정
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # 축 설정
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['</s>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # 매 틱마다 라벨 보여주기
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [70]:
def evaluateAndShowAttention():
    input_tensor = make_alphabet_tensor()
    output_words, attentions = evaluate(
        encoder1, decoder1, input_tensor)
    
    input_sentence = input_tensor[0].tolist()
    input_sentence = list(map(lambda x: i2a[x], input_sentence))
    print('original input =', input_sentence)
    input_length = input_sentence.index('<pad>')
    input_sentence = ' '.join(input_sentence[:input_length])
    print('input  =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

In [71]:
evaluateAndShowAttention()
#evaluateAndShowAttention("elle est trop petit .")
#evaluateAndShowAttention("je ne crains pas de mourir .")
#evaluateAndShowAttention("c est un jeune directeur plein de talent .")

original input = ['m', 'f', 'k', 'e', 'j', 'u', 'n', 'o', 'l', 'b', 'n', 'a', '<pad>', '<pad>', '<pad>']
input  = m f k e j u n o l b n a
output = a n a l o n u j e k f m </s>


C:\Users\jinki.lee\AppData\Local\Temp\ipykernel_14240\1632681039.py:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
C:\Users\jinki.lee\AppData\Local\Temp\ipykernel_14240\1632681039.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
C:\Users\jinki.lee\AppData\Local\Temp\ipykernel_14240\1632681039.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [72]:
generate_random_alphabet_index()

([12, 24, 22, 15, 6, 10, 28, 11, 21, 11, 13], 11)

In [73]:
for d in train_dataloader:
    inputs, targets, lengths = d
    print(inputs)
    print(targets)
    print(lengths)
    print(inputs.shape, targets.shape, lengths.shape)
    break

tensor([[16,  8, 27, 15, 27, 12, 27,  6,  7, 13, 22, 12,  1,  2,  2],
        [27, 18, 10, 14, 23, 10, 22, 22, 28, 21,  4,  4,  1,  2,  2],
        [16, 23, 27, 22, 13,  4,  4, 26,  5,  8,  6, 15,  1,  2,  2],
        [21, 14,  8, 17,  7, 15, 17, 19,  8, 23,  3, 16,  1,  2,  2],
        [15, 28, 25,  4, 23, 25,  6, 25, 26, 21, 18, 19,  1,  2,  2],
        [ 5, 13,  8, 12, 11,  8, 23, 13,  5, 22, 28, 24,  1,  2,  2],
        [26,  3,  6, 17,  9, 15, 12,  6, 15, 19, 18, 18,  1,  2,  2],
        [25, 14, 16, 19, 17, 23,  4, 17, 23, 26, 18,  3,  1,  2,  2],
        [17, 21, 25, 22, 17, 16,  4,  8, 13, 17,  6,  1,  2,  2,  2],
        [ 6,  3, 28,  6, 22, 11, 19, 12, 19, 17,  3,  1,  2,  2,  2],
        [17, 20,  4, 12, 11, 12, 13,  6, 16, 16,  6,  1,  2,  2,  2],
        [18, 14, 27,  7,  7,  9, 19, 10, 18, 27,  5,  1,  2,  2,  2],
        [13,  7, 11, 14, 14, 17, 22, 19, 21, 21,  1,  2,  2,  2,  2],
        [ 8, 11,  6, 22, 17, 13,  5, 13, 25, 24,  1,  2,  2,  2,  2],
        [28,  5,  4,

In [74]:
attn_weight

tensor([[2.8382e-15, 4.4256e-11, 5.1993e-10, 8.4507e-09, 3.3782e-08, 9.9819e-08,
         1.3302e-06, 1.9279e-06, 3.9427e-06, 3.8857e-07, 2.8582e-08, 4.4259e-08,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [5.3788e-19, 1.4025e-13, 4.7627e-15, 5.2463e-15, 3.6329e-15, 1.4115e-12,
         2.0370e-09, 4.9860e-07, 1.6890e-03, 4.5498e-01, 9.9334e-01, 9.9992e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [5.6088e-19, 1.4563e-13, 1.0819e-13, 4.3439e-12, 8.2553e-11, 3.8092e-08,
         2.9317e-05, 2.5778e-03, 4.7160e-01, 5.3829e-01, 6.6560e-03, 8.5008e-05,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.8514e-18, 6.1653e-14, 1.5663e-12, 3.4945e-10, 3.6135e-08, 2.3609e-05,
         1.3353e-02, 4.2087e-01, 5.1690e-01, 6.7063e-03, 6.1794e-06, 3.5629e-08,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [5.4162e-17, 5.3010e-13, 1.0059e-10, 5.8740e-08, 1.7780e-05, 1.4717e-02,
         5.9530e-01, 5.7229e-01, 9.7875e-03, 2.7357e-05, 1.1410e-08, 8.7128e-11,
      

In [ ]:
torch.save(model.state_dict(), PATH)